In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Set the image size and other parameters
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 50

# Define the paths for training and validation directories
train_directory = '/kaggle/input/datasetnetwork11/datasetnetwork1test/train'
valid_directory = '/kaggle/input/datasetnetwork11/datasetnetwork1test/test'

# Load and preprocess the data (no pre-processing, just loading)
def load_image(path):
    image = cv2.imread(path)
    if image is None:
        return None

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

    return image

# Load images from the "No-DR" class
train_data = []
train_labels = []
no_dr_path = os.path.join(train_directory, "0")
for filename in os.listdir(no_dr_path):
    if filename.endswith((".jpeg", ".jpg")):
        image_path = os.path.join(no_dr_path, filename)
        image = load_image(image_path)
        if image is not None:
            train_data.append(image)
            train_labels.append(0)

# Load images from the "DR" class
dr_path = os.path.join(train_directory, "1")
for filename in os.listdir(dr_path):
    if filename.endswith((".jpeg", ".jpg")):
        image_path = os.path.join(dr_path, filename)
        image = load_image(image_path)
        if image is not None:
            train_data.append(image)
            train_labels.append(1)

# Convert the data and labels to NumPy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)

# One-hot encode the labels
train_labels = to_categorical(train_labels)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam



from keras.models import Sequential

# Create the model architecture
model = Sequential([
    Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])


# Define callbacks for early stopping, model checkpoint, and reduce on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/kaggle/working/updated/network1best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

# Compile the model

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
# Load the saved model weights
#model.load_weights('/kaggle/working/updated/best_model.h5')

# Train the model
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    epochs=EPOCHS,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])



In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_data, val_labels)
print("Validation loss:", val_loss)
print("Validation accuracy:", val_accuracy)

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()



In [ ]:
# Make predictions on the validation dataset
predictions = model.predict(val_data)
y_pred = np.argmax(predictions, axis=1)

# Compute precision, recall, accuracy, and F1 score for each class
precision = precision_score(np.argmax(val_labels, axis=1), y_pred, average=None)
recall = recall_score(np.argmax(val_labels, axis=1), y_pred, average=None)
f1 = f1_score(np.argmax(val_labels, axis=1), y_pred, average=None)
accuracy = accuracy_score(np.argmax(val_labels, axis=1), y_pred)

# Display the metrics for each class
class_labels = ['0', '1']  # Update with your class labels

for i, class_label in enumerate(class_labels):
    print(f'Class: {class_label}')
    print(f'Precision: {precision[i]}')
    print(f'Recall: {recall[i]}')
    print(f'F1 Score: {f1[i]}')
    print('---')

# Display overall accuracy
print('Overall Accuracy:', accuracy)







In [ ]:
import numpy as np
from PIL import Image
from keras.models import load_model
image_width = 224
image_height = 224
# Load the saved model
model = load_model('/kaggle/working/updated/network1best_model.h5')

# Load and preprocess the image
image_path = '/kaggle/input/datasetnetwork11/datasetnetwork1test/test/1/007-0725-000.jpg'
image = Image.open(image_path)
image = image.resize((image_width, image_height))  # Adjust the size according to your model's input shape
image = np.array(image)
#image = image / 255.0  # Normalize the image data

# Perform the prediction
predictions = model.predict(np.expand_dims(image, axis=0))
predicted_label = np.argmax(predictions[0])

# Print the predicted label
if predicted_label == 0:
    print("Predicted label: non-retinal")
elif predicted_label == 1:
    print("Predicted label: retinal")
else:
    print("Invalid label predicted")
